In [2]:
import torch

LOAD_TRAIN_PATH = r"D:\Daniel\for_git\LDCT_git\TSCNN\first_stage_code\TrainVal\TRAIN.pth"

LOAD_VAL_PATH = r"D:\Daniel\for_git\LDCT_git\TSCNN\first_stage_code\TrainVal\VAL.pth"

train_data = torch.load(LOAD_TRAIN_PATH)
val_data = torch.load(LOAD_VAL_PATH)

load_train_patches = train_data['patches'] #(N_total, 3, 64, 64) #0~255
load_train_labels = train_data['masks'] #(N_total, 1, 64, 64) #0 or 1

load_val_patches = val_data['patches'] #(N_total, 3, 64, 64) #0~255
load_val_labels = val_data['masks'] #(N_total, 1, 64, 64) #0 or 1

C:\Users\ygz08\AppData\Local\Temp\ipykernel_3428\2084189178.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load(LOAD_TRAIN_PATH)
C:\Users\ygz08\AppDa

In [7]:
if train_data and val_data :
    # 檢查訓練 Patches 維度
    # 確保是四個維度，且 Channel=3, H=64, W=64。N_total 不固定，用 -1 檢查
    assert load_train_patches.dim() == 4, \
        f"訓練 Patches 維度錯誤! 預期 4 維, 實際 {load_train_patches.dim()} 維."
    assert load_train_patches.shape[1:] == (3, 64, 64), \
        f"訓練 Patches Shape 錯誤! 預期 (N, 3, 64, 64), 實際 {load_train_patches.shape}."

    # 檢查訓練 Labels (Masks) 維度
    # 確保是四個維度，且 Channel=1, H=64, W=64
    assert load_train_labels.dim() == 4, \
        f"訓練 Labels 維度錯誤! 預期 4 維, 實際 {load_train_labels.dim()} 維."
    assert load_train_labels.shape[1:] == (1, 64, 64), \
        f"訓練 Labels Shape 錯誤! 預期 (N, 1, 64, 64), 實際 {load_train_labels.shape}."

    # 檢查訓練資料的批次大小是否一致
    assert load_train_patches.shape[0] == load_train_labels.shape[0], \
        f"訓練資料批次大小不一致! Patches: {load_train_patches.shape[0]}, Labels: {load_train_labels.shape[0]}."
        
    # 建議的資料型態檢查
    # Patches (輸入影像) 通常是 float
    # Labels (遮罩) 通常是 float 或 long (如果是類別索引)
    assert load_train_patches.dtype == torch.float32, \
        f"訓練 Patches 資料型態錯誤! 預期 torch.float32, 實際 {load_train_patches.dtype}."
    assert load_train_labels.dtype in [torch.float32, torch.long], \
        f"訓練 Labels 資料型態錯誤! 預期 torch.float32 或 torch.long, 實際 {load_train_labels.dtype}."

    # --- 🎯 檢查驗證資料 (VAL Data Assertions) ---

    # 檢查驗證 Patches 維度
    assert load_val_patches.dim() == 4, \
        f"驗證 Patches 維度錯誤! 預期 4 維, 實際 {load_val_patches.dim()} 維."
    assert load_val_patches.shape[1:] == (3, 64, 64), \
        f"驗證 Patches Shape 錯誤! 預期 (N, 3, 64, 64), 實際 {load_val_patches.shape}."

    # 檢查驗證 Labels (Masks) 維度
    assert load_val_labels.dim() == 4, \
        f"驗證 Labels 維度錯誤! 預期 4 維, 實際 {load_val_labels.dim()} 維."
    assert load_val_labels.shape[1:] == (1, 64, 64), \
        f"驗證 Labels Shape 錯誤! 預期 (N, 1, 64, 64), 實際 {load_val_labels.shape}."
        
    # 檢查驗證資料的批次大小是否一致
    assert load_val_patches.shape[0] == load_val_labels.shape[0], \
        f"驗證資料批次大小不一致! Patches: {load_val_patches.shape[0]}, Labels: {load_val_labels.shape[0]}."

    # 建議的資料型態檢查
    assert load_val_patches.dtype == torch.float32, \
        f"驗證 Patches 資料型態錯誤! 預期 torch.float32, 實際 {load_val_patches.dtype}."
    assert load_val_labels.dtype in [torch.float32, torch.long], \
        f"驗證 Labels 資料型態錯誤! 預期 torch.float32 或 torch.long, 實際 {load_val_labels.dtype}."

    print("✅ Train 有 {} 個樣本，Val 有 {} 個樣本。".format(load_train_patches.shape[0], load_val_patches.shape[0]))
    print("✅ 所有 Tensor 的維度與型態檢查通過！")

✅ Train 有 160 個樣本，Val 有 30 個樣本。
✅ 所有 Tensor 的維度與型態檢查通過！


In [8]:
from torch.utils.data import TensorDataset, DataLoader

# 1. 確保數據類型和範圍正確
# patches (影像) 通常需要是 float 類型，labels (掩碼) 也是 float (因為 sigmoid 輸出)
# 並且將 0~255 的 patches 歸一化到 0~1 範圍 (如果尚未歸一化)

# 假設原始 patches 是 0~255 的 uint8 或 float 類型
# 這裡將其轉為 float 並歸一化 (如果您的預處理還沒做這步)
# 如果您的數據已經是歸一化的 float，可以跳過這一步的除以 255
load_train_patches = load_train_patches.float() / 255.0
load_val_patches = load_val_patches.float() / 255.0

load_train_labels = load_train_labels.float()
load_val_labels = load_val_labels.float()

# 2. 創建 TensorDataset
train_dataset = TensorDataset(load_train_patches, load_train_labels)
val_dataset = TensorDataset(load_val_patches, load_val_labels)

# 3. 創建 DataLoader
BATCH_SIZE = 32 # 可以根據你的 GPU 記憶體調整
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"訓練資料集大小: {len(train_dataset)}")
print(f"訓練批次數量: {len(train_loader)}")

訓練資料集大小: 160
訓練批次數量: 5


In [37]:
import torch.nn.functional as F

# 1. Dice Loss 函數
def dice_loss(pred, target, smooth=1e-6):
    """計算 Dice Loss"""
    # 展平張量
    pred = pred.contiguous().view(-1)
    target = target.contiguous().view(-1)
    
    # 計算交集和總和
    intersection = (pred * target).sum()
    dice = (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)
    
    return 1 - dice

# 2. 總體損失函數 (BCE + Dice)
def combined_loss(pred, target, alpha=0.5):
    """結合 Binary Cross-Entropy Loss 和 Dice Loss"""
    # BCE Loss (適用於 Sigmoid 輸出的二元分割)
    bce = F.binary_cross_entropy(pred, target)
    
    # Dice Loss
    dice = dice_loss(pred, target)
    
    # 結合損失 (alpha 權重可調)
    return alpha * bce + (1 - alpha) * dice
def dice_score(pred, target, smooth=1e-6):
    """計算 Dice 係數 (Dice Coefficient)"""
    # 確保輸入是二元 (0 或 1)
    # 我們將 Sigmoid 輸出 (0-1) 轉換為二元預測 (0/1)，閾值設為 0.5
    pred = (pred > 0.5).float()
    
    # 展平張量
    pred = pred.contiguous().view(-1)
    target = target.contiguous().view(-1)
    
    # 計算交集 (Intersection) 和總和 (Union)
    intersection = (pred * target).sum()
    dice = (2. * intersection + smooth) / (pred.sum() + target.sum() + smooth)
    
    return dice.item()

In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# --- 1. 基礎單元: DenseBlock (DB) ---
class DenseBlock(nn.Module):
    # growth_rate 就是論文中的 n (8, 16, 32, ...)
    def __init__(self, in_channels, growth_rate, dropout_rate=0.2):
        super(DenseBlock, self).__init__()
        
        # 1. 第一個 Conv: BN -> ReLU -> Conv
        self.conv1 = nn.Sequential(
            nn.BatchNorm2d(in_channels), # 期望輸入通道數
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, growth_rate, kernel_size=3, padding=1, bias=False),
            nn.Dropout(dropout_rate) 
        )
        
        # 2. 第二個 Conv: BN -> ReLU -> Conv
        self.conv2 = nn.Sequential(
            nn.BatchNorm2d(growth_rate), # 輸入來自上一個 conv1 的輸出
            nn.ReLU(inplace=True),
            nn.Conv2d(growth_rate, growth_rate, kernel_size=3, padding=1, bias=False),
        )

    def forward(self, x):
        x1 = self.conv1(x) 
        x2 = self.conv2(x1)
        
        # 輸出是 (輸入 x) 和 (最終卷積輸出 x2) 的串聯
        return torch.cat([x, x2], dim=1) 


# --- 2. 堆疊單元: Residual Dense Block (RDB) ---
class ResDenseBlock(nn.Module):
    # RDB 的 in_channels 必須等於它的輸出通道數 (為滿足 U-Net 的跳躍連接要求)
    def __init__(self, in_channels, n_feature, n_layers=6, dropout_rate=0.2):
        super(ResDenseBlock, self).__init__()
        
        self.n_feature = n_feature 
        layers = []
        current_channels = in_channels # 從 RDB 的輸入通道開始
        
        for i in range(n_layers):
            # 每層 DB 的輸入是前一層的總輸出
            db = DenseBlock(current_channels, self.n_feature, dropout_rate)
            layers.append(db)
            # 通道數增加 n_feature
            current_channels += self.n_feature 

        self.dense_blocks = nn.Sequential(*layers)
        
        # 1x1 卷積過渡層 (Transition Layer): 
        # 將總輸出通道數 (current_channels) 轉換回 RDB 期望的輸出通道 (in_channels)，以進行殘差連接
        # 這是滿足 RDB 輸出通道 = 輸入通道的關鍵！
        self.transition_conv = nn.Conv2d(current_channels, in_channels, kernel_size=1, bias=False)

    def forward(self, x):
        dense_output = self.dense_blocks(x)
        processed_output = self.transition_conv(dense_output)
        
        # 殘差連接：輸出 = 壓縮後的密集輸出 + 原始輸入
        return processed_output + x


# --- 3. 整體架構：ResDense UNet (修正版 3) ---
class ResDenseUNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(ResDenseUNet, self).__init__()
        
        # 論文參數 n: [8, 16, 32, 32, 16, 8]
        n_features = [8, 16, 32, 32, 16, 8] 
        
        # --- 通用層 ---
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        def create_upconv(in_c, out_c):
            # 使用 output_padding=1 確保正確的尺寸加倍 (例如 8->16, 16->32)
            return nn.ConvTranspose2d(in_c, out_c, kernel_size=3, stride=2, padding=1, output_padding=1)

        # --- Encoder (縮減路徑) ---
        
        # 1. 初始 Conv (3 -> 8)
        self.init_conv = nn.Conv2d(in_channels, n_features[0], kernel_size=1) 
        self.rdb1 = ResDenseBlock(in_channels=n_features[0], n_feature=n_features[0]) # Output C=8

        # 2. 通道調整 (Max Pool 後): 8 -> 16
        self.trans_conv1 = nn.Conv2d(n_features[0], n_features[1], kernel_size=1) 
        self.rdb2 = ResDenseBlock(in_channels=n_features[1], n_feature=n_features[1]) # Output C=16

        # 3. 通道調整 (Max Pool 後): 16 -> 32
        self.trans_conv2 = nn.Conv2d(n_features[1], n_features[2], kernel_size=1) 
        self.rdb3 = ResDenseBlock(in_channels=n_features[2], n_feature=n_features[2]) # Output C=32 (Bottleneck Input)

        # --- Decoder (擴增路徑) ---
        
        # 4. Stage 4: 8x8 -> 16x16
        # upconv4 輸入=Bottleneck 輸出(32), 輸出=n_features[3]=32 (用於 Concatenate)
        self.upconv4 = create_upconv(n_features[2], n_features[3]) 
        # rdb4 輸入: upconv(32) + skip(32) = 64
        self.rdb4 = ResDenseBlock(in_channels=n_features[3] + n_features[2], n_feature=n_features[3]) # Output C=64
        
        # 5. Stage 5: 16x16 -> 32x32
        # upconv5 輸入=rdb4_out(64), 輸出=n_features[4]=16
        self.upconv5 = create_upconv(n_features[3] + n_features[2], n_features[4])
        # rdb5 輸入: upconv(16) + skip(16) = 32
        self.rdb5 = ResDenseBlock(in_channels=n_features[4] + n_features[1], n_feature=n_features[4]) # Output C=32

        # 6. Stage 6: 32x32 -> 64x64
        # upconv6 輸入=rdb5_out(32), 輸出=n_features[5]=8
        self.upconv6 = create_upconv(n_features[4] + n_features[1], n_features[5])
        # rdb6 輸入: upconv(8) + skip(8) = 16
        self.rdb6 = ResDenseBlock(in_channels=n_features[5] + n_features[0], n_feature=n_features[5]) # Output C=16

        # --- 最終輸出層 (16 -> 1) ---
        self.final_conv = nn.Conv2d(n_features[5] + n_features[0], out_channels, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        
        # --- 打印 Shape 以供檢查 ---
        # print(f"Input: {x.shape}")
        
        # 1. Initial Conv & RDB1
        x = self.init_conv(x) 
        rdb1_out = self.rdb1(x) # Skip 1 (C=8, 64x64)
        
        # 2. Stage 2
        p1 = self.pool(rdb1_out) 
        t1 = self.trans_conv1(p1) # C: 8 -> 16
        rdb2_out = self.rdb2(t1) # Skip 2 (C=16, 32x32)
        
        # 3. Stage 3
        p2 = self.pool(rdb2_out)
        t2 = self.trans_conv2(p2) # C: 16 -> 32
        rdb3_out = self.rdb3(t2) # Skip 3 (C=32, 16x16)
        
        # 4. Bottleneck
        p3 = self.pool(rdb3_out) # Bottleneck (C=32, 8x8)

        # --- Decoder ---
        
        # 5. Stage 4: 8x8 -> 16x16
        up4 = self.upconv4(p3) # C=32
        cat4 = torch.cat([up4, rdb3_out], dim=1) # C=64
        rdb4_out = self.rdb4(cat4) # C=64
        
        # 6. Stage 5: 16x16 -> 32x32
        up5 = self.upconv5(rdb4_out) # C=16
        cat5 = torch.cat([up5, rdb2_out], dim=1) # C=32
        rdb5_out = self.rdb5(cat5) # C=32

        # 7. Stage 6: 32x32 -> 64x64
        up6 = self.upconv6(rdb5_out) # C=8
        cat6 = torch.cat([up6, rdb1_out], dim=1) # C=16
        rdb6_out = self.rdb6(cat6) # C=16

        # 8. Output
        return self.sigmoid(self.final_conv(rdb6_out))

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# 遵守論文設置的參數
BATCH_SIZE = 64
NUM_EPOCHS = 15 # 總訓練代數
INITIAL_LR = 0.0001
FINE_TUNE_LR = 0.00001 # 雖然論文未明確說明何時進行 Fine-tuning，但我們將使用學習率調度來模擬

def train_model_optimized(model, train_loader, val_loader, optimizer, num_epochs, device):
    
    # 初始化學習率調度器，模擬學習率下降機制 (例如在性能停滯時降低 LR)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose=True)
    
    best_dice = 0.0
    epochs_no_improve = 0
    early_stop_patience = 5 # 論文規定的早停寬限期
    
    print(f"訓練開始：Batch Size={BATCH_SIZE}, LR={optimizer.param_groups[0]['lr']}")
    
    for epoch in range(num_epochs):
        # ------------------ 訓練階段 ------------------
        model.train() 
        train_loss_sum = 0
        
        for inputs, masks in train_loader:
            inputs, masks = inputs.to(device), masks.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = combined_loss(outputs, masks) 
            
            loss.backward()
            optimizer.step()
            
            train_loss_sum += loss.item() * inputs.size(0)
            
        train_loss_avg = train_loss_sum / len(train_loader.dataset)
        
        # ------------------ 驗證階段 ------------------
        model.eval() 
        val_loss_sum = 0
        val_dice_sum = 0
        
        with torch.no_grad():
            for inputs, masks in val_loader:
                inputs, masks = inputs.to(device), masks.to(device)
                outputs = model(inputs)
                
                loss = combined_loss(outputs, masks)
                val_loss_sum += loss.item() * inputs.size(0)
                
                # 計算 Dice 係數
                val_dice_sum += dice_score(outputs, masks) * inputs.size(0)
                
        val_loss_avg = val_loss_sum / len(val_loader.dataset)
        val_dice_avg = val_dice_sum / len(val_loader.dataset)
        
        print(f"Epoch {epoch+1}/{num_epochs}: Train Loss={train_loss_avg:.4f}, "
              f"Val Loss={val_loss_avg:.4f}, Val DICE Score={val_dice_avg:.4f}")
        
        # ------------------ 早停與學習率調度 ------------------
        
        # 根據 Dice Score 調整學習率
        scheduler.step(val_dice_avg)
        
        # 檢查 Dice Score 是否提升
        if val_dice_avg > best_dice:
            best_dice = val_dice_avg
            epochs_no_improve = 0
            # 💡 最佳模型權重保存 (Checkpointing)
            torch.save(model.state_dict(), 'pth/best_stage1_unet.pth')
            print("🚀 Dice Score 提升！保存最佳模型。")
        else:
            epochs_no_improve += 1
            print(f"性能未提升。已停滯 {epochs_no_improve} epochs。")
            
        # 執行早停策略
        if epochs_no_improve >= early_stop_patience:
            print(f"🚨 早停觸發！驗證性能連續 {early_stop_patience} epochs 未提升。")
            # 設置新的學習率，模擬論文中的 Fine-tuning 步驟
            for param_group in optimizer.param_groups:
                if param_group['lr'] > FINE_TUNE_LR:
                     param_group['lr'] = FINE_TUNE_LR
                     print(f"✅ 學習率已降至 Fine-tuning LR: {FINE_TUNE_LR}")
                     epochs_no_improve = 0 # 重置計數器，讓模型有 5 次機會適應新 LR
                     early_stop_patience = 5 # 保持 5 次寬限期
                else:
                    # 如果已經是最低 LR，則完全停止訓練
                    print("⚠️ Fine-tuning LR 已應用，但仍未改善，訓練完全停止。")
                    return 

    print("總訓練代數已完成。")

In [38]:


BATCH_SIZE = 64 # 遵守論文設置
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# --- 模型與優化器設定 ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResDenseUNet(in_channels=3, out_channels=1).to(device)

# 使用論文設置的初始學習率
optimizer = torch.optim.Adam(model.parameters(), lr=INITIAL_LR)

# --- 啟動訓練 ---
train_model_optimized(model, train_loader, val_loader, optimizer, NUM_EPOCHS, device)

訓練開始：Batch Size=64, LR=0.0001
Epoch 1/15: Train Loss=0.8882, Val Loss=0.6558, Val DICE Score=0.5280
🚀 Dice Score 提升！保存最佳模型。
Epoch 2/15: Train Loss=0.8703, Val Loss=0.6546, Val DICE Score=0.5638
🚀 Dice Score 提升！保存最佳模型。
Epoch 3/15: Train Loss=0.8533, Val Loss=0.6528, Val DICE Score=0.6054
🚀 Dice Score 提升！保存最佳模型。
Epoch 4/15: Train Loss=0.8367, Val Loss=0.6503, Val DICE Score=0.6487
🚀 Dice Score 提升！保存最佳模型。
Epoch 5/15: Train Loss=0.8193, Val Loss=0.6471, Val DICE Score=0.6833
🚀 Dice Score 提升！保存最佳模型。
Epoch 6/15: Train Loss=0.8015, Val Loss=0.6433, Val DICE Score=0.6389
性能未提升。已停滯 1 epochs。
Epoch 7/15: Train Loss=0.7829, Val Loss=0.6396, Val DICE Score=0.5855
性能未提升。已停滯 2 epochs。
Epoch 8/15: Train Loss=0.7632, Val Loss=0.6368, Val DICE Score=0.4424
性能未提升。已停滯 3 epochs。
Epoch 9/15: Train Loss=0.7426, Val Loss=0.6360, Val DICE Score=0.1957
性能未提升。已停滯 4 epochs。
Epoch 10/15: Train Loss=0.7213, Val Loss=0.6383, Val DICE Score=0.0242
性能未提升。已停滯 5 epochs。
🚨 早停觸發！驗證性能連續 5 epochs 未提升。
✅ 學習率已降至 Fine-tuning 